In [ ]:
# 1) ENV: установить переменные до любых heavy импортов (выполните сразу)
import os, importlib

_changed = False

def _set(k,v):
    global _changed
    if os.environ.get(k) != v:
        os.environ[k] = v
        _changed = True
# подавляем XLA/TensorFlow шумы
_set('TF_CPP_MIN_LOG_LEVEL','3')
_set('XLA_PYTHON_CLIENT_PREALLOCATE','false')
_set('XLA_PYTHON_CLIENT_MEM_FRACTION','0.0')
_set('JAX_PLATFORM_NAME','cpu')
_set('JAX_PLATFORMS','cpu')
print('ENV set. Restart kernel AFTER you change these and BEFORE importing heavy libs if _changed = True')
print('changed_env =', _changed)

In [ ]:
# 2) KAGGLE checks: basic hints and GPU info
import os, sys, importlib, subprocess, json
print('Running in Kaggle:', any(p.startswith('/kaggle') for p in (os.getcwd(),)) or os.environ.get('KAGGLE_KERNEL_RUN_TYPE') is not None)
# lightweight torch check (subprocess to avoid heavy import delays)
try:
    cmd = [sys.executable, '-c', 'import torch, json; print(json.dumps({"cuda": torch.cuda.is_available(), "device_count": torch.cuda.device_count(), "name": (torch.cuda.get_device_name(0).strip() if torch.cuda.is_available() else "no gpu")}))']
    res = subprocess.run(cmd, capture_output=True, text=True, timeout=12)
    print('torch probe:', res.stdout.strip())
except Exception as e:
    print('Torch probe skipped:', e)
# HF token hint
print('HF token present in env:', bool(os.environ.get('HUGGINGFACE_HUB_TOKEN') or os.environ.get('HF_TOKEN')))

In [ ]:
# 3) Optional: install deps (uncomment to run)
# Для ускорения повторных запусков — выполняйте только при отсутствии пакетов
# !pip install -q diffusers[torch] transformers accelerate imageio imageio-ffmpeg ipywidgets opencv-python
# Если планируете запуск Real-ESRGAN:
# !pip install -q basicsr facexlib gfpgan realesrgan
print('Deps: оставлены как опция — устанавливайте вручную при необходимости')

# Prompts — редактируемая часть (меняйте только здесь)

In [ ]:
# 4) Prompts: сохранение и короткая GUI fallback
import json, os
PROMPTS = {
  'BASE_PROMPT': 'cinematic portrait of gojo satoru, white spiky hair, black blindfold, confident expression, anime style, highly detailed',
  'MOTION_PROMPT': 'turning head, hair flowing, smooth motion',
  'EXTRA_PROMPT': 'dramatic rim lighting, soft bloom',
  'NEGATIVE_PROMPT': 'blurry, deformed, watermark, text, bad anatomy'
}
with open('prompts.json','w',encoding='utf-8') as f:
    json.dump(PROMPTS,f,ensure_ascii=False,indent=2)
print('Saved prompts.json (edit PROMPTS dict above to change)')

# Генерация (AnimateDiff или SDXL) — запустите только нужную ячейку

In [ ]:
# 5a) AnimateDiff generation (fast path) — включите/настройте флаги и запустите
USE_ANIMATEDIFF = True
NUM_FRAMES = 8
WIDTH = 512
HEIGHT = 768
STEPS = 20
from pathlib import Path
FRAMES_DIR = Path('/kaggle/working/frames')
FRAMES_DIR.mkdir(parents=True, exist_ok=True)
if USE_ANIMATEDIFF:
    try:
        from diffusers import AnimateDiffPipeline, MotionAdapter, EulerDiscreteScheduler
        import torch
        adapter = MotionAdapter.from_pretrained('guoyww/animatediff-motion-adapter-v1-5-2', torch_dtype=torch.float16)
        pipe = AnimateDiffPipeline.from_pretrained('runwayml/stable-diffusion-v1-5', motion_adapter=adapter, torch_dtype=torch.float16).to('cuda')
        pipe.scheduler = EulerDiscreteScheduler.from_config(pipe.scheduler.config)
        pipe.enable_vae_slicing(); pipe.enable_model_cpu_offload()
        out = pipe(prompt= (open('prompts.json','r',encoding='utf-8').read()), num_frames=NUM_FRAMES, width=WIDTH, height=HEIGHT, num_inference_steps=STEPS, guidance_scale=7.5, generator=torch.Generator('cuda').manual_seed(42))
        frames = out.frames[0]
        for i,im in enumerate(frames):
            im.save(FRAMES_DIR / f'{i}.png')
        print('Saved', len(frames), 'frames to', FRAMES_DIR)
    except Exception as e:
        print('AnimateDiff failed:', e)
else:
    print('AnimateDiff disabled — run SDXL cell вместо')

In [ ]:
# 5b) SDXL static frame generation (alternative) — uncomment to use instead of AnimateDiff
# from diffusers import StableDiffusionXLPipeline
# import torch
# pipe = StableDiffusionXLPipeline.from_pretrained('stabilityai/stable-diffusion-xl-base-1.0', torch_dtype=torch.float16).to('cuda')
# pipe.enable_attention_slicing(); pipe.enable_vae_slicing()
# for i in range(NUM_FRAMES):
#     img = pipe(prompt=... ).images[0]; img.save(FRAMES_DIR / f'{i}.png')
# print('Saved static frames')
print('SDXL cell ready as alternative — uncomment to run')

# Real-ESRGAN upscaling — надежная подготовка shim и запуск

In [ ]:
# 6) Robust shim writer (writes to /kaggle/working/Real-ESRGAN/inference_with_shim_full.py or falls back)
import os, textwrap, py_compile
OUT_DIR = '/kaggle/working/Real-ESRGAN'
OUT_PATH = os.path.join(OUT_DIR, 'inference_with_shim_full.py')
SHIM = textwrap.dedent('''
    import sys
    import os
    import types
    import re
    import runpy
    
    try:
        import torchvision.transforms.functional_tensor as _ft
    except Exception:
        try:
            import torchvision.transforms.functional as _f
            mod_ft = types.ModuleType('torchvision.transforms.functional_tensor')
            mod_ft.rgb_to_grayscale = getattr(_f, 'rgb_to_grayscale', None)
            mod_ft.convert_image_dtype = getattr(_f, 'convert_image_dtype', None)
            sys.modules['torchvision.transforms.functional_tensor'] = mod_ft
        except Exception:
            mod_ft = types.ModuleType('torchvision.transforms.functional_tensor')
            def _rgb_to_grayscale(x):
                raise ImportError('rgb_to_grayscale not available')
            def _convert_image_dtype(x, dtype):
                raise ImportError('convert_image_dtype not available')
            mod_ft.rgb_to_grayscale = _rgb_to_grayscale
            mod_ft.convert_image_dtype = _convert_image_dtype
            sys.modules['torchvision.transforms.functional_tensor'] = mod_ft
    
    if 'torchvision.utils' not in sys.modules:
        mod_utils = types.ModuleType('torchvision.utils')
        def make_grid(x, nrow=8, padding=2, normalize=False, range=None, scale_each=False, pad_value=0):
            try:
                if isinstance(x, (list, tuple)) and len(x) > 0:
                    return x[0]
                return x
            except Exception:
                return x
        mod_utils.make_grid = make_grid
        sys.modules['torchvision.utils'] = mod_utils
    
    if 'realesrgan.version' not in sys.modules:
        vermod = types.ModuleType('realesrgan.version')
        vermod.__version__ = '0.3.0'
        vermod.__all__ = ['__version__']
        sys.modules['realesrgan.version'] = vermod
    
    def _infer_netscale(argv):
        for i, a in enumerate(argv):
            if a in ('-n', '--name', '--model') and i+1 < len(argv):
                m = re.match(r"(\\d+)x", argv[i+1])
                if m:
                    try:
                        return int(m.group(1))
                    except Exception:
                        pass
        for i, a in enumerate(argv):
            if a in ('-s', '--scale') and i+1 < len(argv):
                try:
                    return int(argv[i+1])
                except Exception:
                    pass
        return 4
    
    def _detect_weight_file():
        try:
            wdir = os.path.join(os.getcwd(), 'weights')
            if not os.path.isdir(wdir):
                return None
            exts = ('.pth', '.pt', '.safetensors')
            candidates = [f for f in os.listdir(wdir) if f.lower().endswith(exts)]
            if not candidates:
                return None
            for name in candidates:
                if '4x' in name.lower() or 'ultrasharp' in name.lower():
                    return os.path.join(wdir, name)
            return os.path.join(wdir, candidates[0])
        except Exception:
            return None
    
    if __name__ == '__main__':
        argv = sys.argv[1:]
        netscale = _infer_netscale(argv)
        detected_model = _detect_weight_file()
        script_path = os.path.join(os.getcwd(), 'inference_realesrgan.py')
        try:
            with open(script_path, 'r', encoding='utf-8') as _f:
                _code = _f.read()
        except Exception:
            sys.argv = [script_path] + argv
            runpy.run_path('inference_realesrgan.py', run_name='__main__')
        else:
            _globals = {'__name__': '__main__', '__file__': script_path, 'netscale': netscale, 'model': detected_model, 'outscale': netscale, 'scale': netscale}
            _globals['sys'] = sys
            sys.argv = [script_path] + argv
            exec(compile(_code, script_path, 'exec'), _globals)
    ''')
# write shim with safe fallbacks
try:
    os.makedirs(OUT_DIR, exist_ok=True)
    with open(OUT_PATH, 'w', encoding='utf-8', newline='\n') as f:
        f.write(SHIM)
    py_compile.compile(OUT_PATH, doraise=True)
    print('Shim written and compiled:', OUT_PATH)
except Exception as e:
    fallback = os.path.join(os.getcwd(), 'inference_with_shim_full.py')
    with open(fallback, 'w', encoding='utf-8') as f:
        f.write(SHIM)
    print('Fallback shim written to', fallback, 'due to', e)

In [ ]:
# 7) Prepare Real-ESRGAN repo, copy weights and prepare run_cmd (do not auto-run)
import os, shutil, subprocess, sys
WORKSPACE = '/kaggle/working'
REPO_DIR = os.path.join(WORKSPACE, 'Real-ESRGAN')
UPSCALE_MODEL_NAME = '4x-UltraSharp.pth'
# candidate weight in datasets (adjust to your dataset mount)
c1 = os.path.join('/kaggle/input/comfyui-models-gojo', UPSCALE_MODEL_NAME)
weights_copied = False
if os.path.exists(c1) and os.path.exists(REPO_DIR):
    try:
        wd = os.path.join(REPO_DIR, 'weights')
        os.makedirs(wd, exist_ok=True)
        shutil.copy(c1, wd)
        weights_copied = True
        print('Copied weight to', wd)
    except Exception as e:
        print('Failed to copy weight:', e)
else:
    print('Either dataset weight or Real-ESRGAN repo missing; you may need to clone or attach dataset')
# Build run_cmd (prefer running the real script directly to avoid shim exec quirks)
shim_path = os.path.join(REPO_DIR, 'inference_with_shim_full.py')
script_path = os.path.join(REPO_DIR, 'inference_realesrgan.py')
run_cwd = None
if os.path.exists(script_path):
    # run the actual script directly (safest for argument parsing and avoids UnboundLocalError from exec)
    run_cmd = [sys.executable, script_path, '-n', '4x-UltraSharp', '-i', str(FRAMES_DIR), '-o', str(FRAMES_DIR) + '_upscaled', '--fp32', '--outscale', '4']
    run_cwd = REPO_DIR
elif os.path.exists(shim_path):
    # fallback to shim if script not present
    run_cmd = [sys.executable, shim_path, '-n', '4x-UltraSharp', '-i', str(FRAMES_DIR), '-o', str(FRAMES_DIR) + '_upscaled', '--fp32', '--outscale', '4']
    run_cwd = REPO_DIR
else:
    run_cmd = None
    run_cwd = None
print('Prepared run_cmd:', run_cmd, 'run_cwd:', run_cwd)

In [ ]:
# 8) Debug + optional retry runner for Real-ESRGAN (replacement for fragile cells)
import os, subprocess, time, shutil

# User toggle: set to True to actually run the prepared command(s)
retry_now = True  # <<< set to False to do a dry-run; currently enabled to run injector/shim when you execute this cell

def tail(path, max_chars=8000):
    try:
        with open(path, 'r', encoding='utf-8', errors='replace') as f:
            data = f.read()
        return data[-max_chars:]
    except Exception:
        return ''

INFERENCE_LOG = os.path.join('/kaggle/working', 'logs', 'real_esrgan_inference.log')
os.makedirs(os.path.dirname(INFERENCE_LOG), exist_ok=True)
print('INFERENCE_LOG ->', INFERENCE_LOG)

# Ensure run_cwd points to REPO_DIR if possible
if not globals().get('run_cwd') and os.path.exists(REPO_DIR) and os.path.exists(os.path.join(REPO_DIR, 'inference_realesrgan.py')):
    run_cwd = REPO_DIR
    print('Auto-set run_cwd ->', run_cwd)

# If inference_realesrgan.py is missing in run_cwd, attempt to fetch it automatically (clone or download)
if run_cwd and not os.path.exists(os.path.join(run_cwd, 'inference_realesrgan.py')):
    print('inference_realesrgan.py not found in', run_cwd, '-> attempting to fetch from upstream Real-ESRGAN repo')
    try:
        tmp_dir = os.path.join(WORKSPACE, 'Real-ESRGAN_tmp_clone')
        if os.path.exists(tmp_dir):
            try: shutil.rmtree(tmp_dir)
            except Exception: pass
        print('Cloning Real-ESRGAN into temporary dir:', tmp_dir)
        r = subprocess.run(['git', 'clone', '--depth', '1', 'https://github.com/xinntao/Real-ESRGAN', tmp_dir], capture_output=True, text=True, timeout=240)
        if r.returncode == 0 and os.path.exists(os.path.join(tmp_dir, 'inference_realesrgan.py')):
            shutil.copy(os.path.join(tmp_dir, 'inference_realesrgan.py'), os.path.join(run_cwd, 'inference_realesrgan.py'))
            print('Copied inference_realesrgan.py into', run_cwd)
            try: shutil.rmtree(tmp_dir)
            except Exception: pass
        else:
            print('Clone did not produce inference_realesrgan.py or clone failed:', r.returncode)
            print('git stderr (first 400 chars):', (r.stderr or '')[:400])
            # Try downloading raw file as fallback
            raw_url = 'https://raw.githubusercontent.com/xinntao/Real-ESRGAN/master/inference_realesrgan.py'
            out_path = os.path.join(run_cwd, 'inference_realesrgan.py')
            try:
                print('Attempting to download raw file from', raw_url)
                d = subprocess.run(['curl', '-fsSL', raw_url, '-o', out_path], capture_output=True, text=True, timeout=30)
                if d.returncode == 0 and os.path.exists(out_path):
                    print('Downloaded inference_realesrgan.py to', out_path)
                else:
                    print('curl failed or file not present after download. rc=', d.returncode, 'stderr=', (d.stderr or '')[:400])
            except Exception as e:
                print('curl attempt failed:', e)
    except Exception as e:
        print('Auto-fetch attempt failed:', e)

# Summary before running
print('Prepared run_cmd:', run_cmd)
print('run_cwd:', run_cwd)

# Execution + robust retry
if retry_now:
    print('Running run_cmd...')
    start = time.time()
    try:
        # Prepare compat root and wrapper
        if run_cwd:
            compat_root = os.path.join(run_cwd, 'compat_site')
            wrapper_path = os.path.join(run_cwd, 'run_with_compat.py')
        else:
            compat_root = os.path.join(os.getcwd(), 'compat_site')
            wrapper_path = os.path.join(os.getcwd(), 'run_with_compat.py')

        # ensure compat package exists (may have been created earlier)
        try:
            os.makedirs(compat_root, exist_ok=True)
        except Exception:
            pass

        # write wrapper that injects compat_root and provides a synthetic functional_tensor module if needed
        wrapper_code = f'''import sys, os, runpy, types
# Insert compat_site at front so imports see our shim
compat = os.path.join(os.path.dirname(__file__), 'compat_site')
if os.path.exists(compat) and compat not in sys.path:
    sys.path.insert(0, compat)
# Ensure a minimal torchvision.transforms.functional_tensor module exists in sys.modules before imports
try:
    import importlib
    spec = importlib.util.find_spec('torchvision.transforms.functional_tensor')
except Exception:
    spec = None
if spec is None:
    try:
        # attempt to create a synthetic module using torchvision.transforms.functional
        from torchvision import transforms as _trans
        try:
            _func = _trans.functional
        except Exception:
            _func = None
        mod = types.ModuleType('torchvision.transforms.functional_tensor')
        if _func is not None:
            rgb = getattr(_func, 'rgb_to_grayscale', None)
            conv = getattr(_func, 'convert_image_dtype', None)
            if rgb is not None:
                mod.rgb_to_grayscale = rgb
            if conv is not None:
                mod.convert_image_dtype = conv
        # fallback placeholders
        if not hasattr(mod, 'rgb_to_grayscale'):
            def _rgb_fail(x):
                raise ImportError('rgb_to_grayscale not available')
            mod.rgb_to_grayscale = _rgb_fail
        if not hasattr(mod, 'convert_image_dtype'):
            def _conv_fail(x, dtype):
                raise ImportError('convert_image_dtype not available')
            mod.convert_image_dtype = _conv_fail
        import sys as _sys
        _sys.modules['torchvision.transforms.functional_tensor'] = mod
    except Exception:
        pass
# Determine script to run from argv[1]
if len(sys.argv) < 2:
    print('run_with_compat: missing script argument', file=sys.stderr)
    sys.exit(2)
script = sys.argv[1]
# propagate CLI args: keep script as argv[0]
sys.argv = [script] + sys.argv[2:]
runpy.run_path(script, run_name='__main__')
'''
        try:
            with open(wrapper_path, 'w', encoding='utf-8') as wf:
                wf.write(wrapper_code)
            os.chmod(wrapper_path, 0o755)
            print('Wrote wrapper script to', wrapper_path)
        except Exception as _e:
            print('Could not write wrapper script:', _e)

        # write an injector script that preloads compat and injects a synthetic module before running the script
        try:
            if run_cwd:
                inject_path = os.path.join(run_cwd, 'inject_and_exec.py')
            else:
                inject_path = os.path.join(os.getcwd(), 'inject_and_exec.py')
            inject_code = '''import sys, os, runpy, types, re
# insert compat_site at front
compat = os.path.join(os.path.dirname(__file__), 'compat_site')
if os.path.exists(compat) and compat not in sys.path:
    sys.path.insert(0, compat)
# ensure sys.modules has torchvision.transforms.functional_tensor to bypass package-level lookup
modname = 'torchvision.transforms.functional_tensor'
if modname not in sys.modules:
    try:
        from torchvision import transforms as _trans
        try:
            _func = _trans.functional
        except Exception:
            _func = None
        mod = types.ModuleType(modname)
        if _func is not None:
            rgb = getattr(_func, 'rgb_to_grayscale', None)
            conv = getattr(_func, 'convert_image_dtype', None)
            if rgb is not None:
                mod.rgb_to_grayscale = rgb
            if conv is not None:
                mod.convert_image_dtype = conv
        # fallbacks
        if not hasattr(mod, 'rgb_to_grayscale'):
            def _rgb_fail(x):
                raise ImportError('rgb_to_grayscale not available')
            mod.rgb_to_grayscale = _rgb_fail
        if not hasattr(mod, 'convert_image_dtype'):
            def _conv_fail(x, dtype):
                raise ImportError('convert_image_dtype not available')
            mod.convert_image_dtype = _conv_fail
        sys.modules[modname] = mod
    except Exception:
        pass

# get target script and args
if len(sys.argv) < 2:
    print('inject_and_exec: missing script argument', file=sys.stderr)
    sys.exit(2)
script = sys.argv[1]
args = sys.argv[2:]
base = os.path.basename(script)

if base == 'inference_realesrgan.py':
    # infer netscale from args (look for -n NAME or -s SCALE) or env
    joined = [script] + args
    netscale = None
    for i, a in enumerate(joined):
        if a in ('-n', '--name', '--model') and i+1 < len(joined):
            m = re.match(r"(\d+)x", joined[i+1])
            if m:
                try:
                    netscale = int(m.group(1))
                    break
                except Exception:
                    pass
    if netscale is None:
        for i, a in enumerate(joined):
            if a in ('-s', '--scale') and i+1 < len(joined):
                try:
                    netscale = int(joined[i+1])
                    break
                except Exception:
                    pass
    if netscale is None:
        netscale = int(os.environ.get('NETSCALE') or os.environ.get('SCALE') or 4)
    # read and exec with prepared globals so netscale is defined
    try:
        with open(script, 'r', encoding='utf-8') as f:
            code = f.read()
        _globals = {'__name__': '__main__', '__file__': script, 'netscale': netscale, 'scale': netscale, 'outscale': netscale}
        _globals['sys'] = sys
        sys.argv = [script] + args
        exec(compile(code, script, 'exec'), _globals)
    except Exception:
        # fallback to runpy if exec fails
        sys.argv = [script] + args
        runpy.run_path(script, run_name='__main__')
else:
    # generic path: just run
    sys.argv = [script] + args
    runpy.run_path(script, run_name='__main__')
'''
            with open(inject_path, 'w', encoding='utf-8') as inf:
                inf.write(inject_code)
            os.chmod(inject_path, 0o755)
            print('Wrote injector script to', inject_path)
        except Exception as _e:
            print('Could not write injector script:', _e)

        # Decide command to execute: prefer wrapper so compat is injected before any import
        exec_cmd = None
        if run_cmd and len(run_cmd) >= 2 and (run_cmd[1].endswith('inference_realesrgan.py') or run_cmd[1].endswith('inference_with_shim_full.py')) and os.path.exists(wrapper_path):
            # pass the target script path as the first argument to wrapper
            exec_cmd = [run_cmd[0], wrapper_path, run_cmd[1]] + run_cmd[2:]
        else:
            exec_cmd = run_cmd

        # Force execution to use the injector script so module injection happens before imports
        exec_cmd = None
        if run_cmd and len(run_cmd) >= 2 and (run_cmd[1].endswith('inference_realesrgan.py') or run_cmd[1].endswith('inference_with_shim_full.py')) and os.path.exists(inject_path):
            # build command: [python, inject_and_exec.py, target_script] + original args after the script
            exec_cmd = [run_cmd[0], inject_path, run_cmd[1]] + run_cmd[2:]
        else:
            exec_cmd = run_cmd

        # Run process with env and cwd
        if run_cwd:
            res = subprocess.run(exec_cmd, capture_output=True, text=True, timeout=3600, cwd=run_cwd, env=env)
        else:
            res = subprocess.run(exec_cmd, capture_output=True, text=True, timeout=3600, env=env)

        dur = time.time() - start
        print('Return code =', res.returncode, 'duration=', dur)
        stderr_tail = (res.stderr or '')[:8000]
        with open(INFERENCE_LOG, 'a', encoding='utf-8') as lf:
            lf.write('\n=== ATTEMPT STDOUT ===\n')
            lf.write(res.stdout or '')
            lf.write('\n=== ATTEMPT STDERR ===\n')
            lf.write(res.stderr or '')
        print('\n--- subprocess stderr tail ---\n')
        print(stderr_tail)
        globals()['last_inference_stderr'] = res.stderr

        # Determine whether a retry is needed
        need_retry = False
        if res.returncode != 0:
            err = (res.stderr or '').lower()
            if 'unboundlocalerror' in err or 'netscale' in err or 'local variable' in err:
                need_retry = True
            if 'no module named "torchvision.transforms.functional_tensor"' in err or 'no module named \'torchvision.transforms.functional_tensor\'' in err or ('functional_tensor' in err and 'module' in err):
                need_retry = True

        # Retry path with augmented flags and same env
        if need_retry:
            print('\nDetected netscale/unboundlocal error or missing functional_tensor; retrying with explicit flags and env override...')
            aug_cmd = list(run_cmd)
            if '-s' not in aug_cmd:
                aug_cmd += ['-s', '4']
            if '--outscale' not in aug_cmd:
                aug_cmd += ['--outscale', '4']
            env.update({'NETSCALE': '4', 'SCALE': '4', 'OUTSCALE': '4'})
            print('Retry command:', aug_cmd)
            try:
                # If injector is available, run aug_cmd through it to ensure compat shim and netscale injection
                if run_cwd:
                    possible_inject = os.path.join(run_cwd, 'inject_and_exec.py')
                else:
                    possible_inject = os.path.join(os.getcwd(), 'inject_and_exec.py')
                if os.path.exists(possible_inject) and len(aug_cmd) >= 2:
                    target_script = aug_cmd[1]
                    augmented_args = aug_cmd[2:]
                    exec_aug_cmd = [aug_cmd[0], possible_inject, target_script] + augmented_args
                else:
                    exec_aug_cmd = aug_cmd
                if run_cwd:
                    res2 = subprocess.run(exec_aug_cmd, capture_output=True, text=True, timeout=3600, cwd=run_cwd, env=env)
                else:
                    res2 = subprocess.run(exec_aug_cmd, capture_output=True, text=True, timeout=3600, env=env)
                dur2 = time.time() - start
                print('Retry return code =', res2.returncode, 'duration=', dur2)
                with open(INFERENCE_LOG, 'a', encoding='utf-8') as lf:
                    lf.write('\n=== RETRY STDOUT ===\n')
                    lf.write(res2.stdout or '')
                    lf.write('\n=== RETRY STDERR ===\n')
                    lf.write(res2.stderr or '')
                print('\n--- retry stderr tail ---\n')
                print((res2.stderr or '')[:8000])
                globals()['last_inference_stderr'] = res2.stderr
            except Exception as e2:
                print('Retry attempt failed to execute:', e2)
    except Exception as e:
        print('Run failed:', e)
else:
    print('Dry-run: not executing. Set retry_now = True and re-run this cell to execute the prepared run_cmd.')

In [ ]:
# 9) Move upscaled frames into frames dir if present (safe rename)
import os, shutil, glob
UP_DIR = str(FRAMES_DIR) + '_upscaled'
if os.path.exists(UP_DIR):
    print('Found upscaled dir -> moving into frames')
    try:
        # remove old frames (be careful)
        shutil.rmtree(str(FRAMES_DIR), ignore_errors=True)
        shutil.move(UP_DIR, str(FRAMES_DIR))
        # optional rename of *_out.png to index.png
        ups = sorted(glob.glob(str(FRAMES_DIR) + '/*'))
        for i,p in enumerate(ups):
            try: os.rename(p, os.path.join(str(FRAMES_DIR), f'{i}.png'))
            except Exception: pass
        print('Moved and normalized upscaled frames')
    except Exception as e:
        print('Error moving upscaled frames:', e)
else:
    print('No upscaled frames dir found (run Real-ESRGAN first)')

# Завершение — краткий отчет

In [ ]:
print('Workflow ready.\n- Generate frames (AnimateDiff or SDXL).\n- Optionally write/verify shim (cell 6).\n- Prepare repo and weights (cell 7).\n- Run upscale with run_cmd (cell 8, set retry_now=True).')

In [ ]:
# === AUTO: install + write shim + run (one-click, safe by default) ===
# Usage: set confirm=True and other flags below, then run this cell.
import os, sys, subprocess, textwrap, time, shutil
from pathlib import Path
# --- User-configurable flags (safe defaults) ---
confirm = False        # must be True to perform actions
install = False        # whether to pip install required packages
run = False            # whether to run the inference after setup
restart_kernel = False # if True, attempt to request a kernel restart at the end (may require manual confirm)
# packages to install when install=True
deps = ['basicsr', 'facexlib', 'gfpgan', 'realesrgan']
# Paths and args (adjust if needed)
WORKSPACE = '/kaggle/working'
REPO_DIR = os.path.join(WORKSPACE, 'Real-ESRGAN')
SHIM_PATH = os.path.join(REPO_DIR, 'inference_with_shim_full.py')
FALLBACK_SHIM = os.path.join(os.getcwd(), 'inference_with_shim_full.py')
INFERENCE_LOG = os.path.join(WORKSPACE, 'logs', 'real_esrgan_inference.log')
os.makedirs(os.path.dirname(INFERENCE_LOG), exist_ok=True)
# Default run args for shim (modify before confirm=True if needed)
frames_dir = str(Path(WORKSPACE) / 'frames')
run_args = ['-n', '4x-UltraSharp', '-i', frames_dir, '-o', frames_dir + '_upscaled', '--fp32', '--outscale', '4']
# Shim content (kept minimal and safe; generated via dedent)
SHIM = textwrap.dedent('''
    import sys
    import os
    import types
    import re
    import runpy

    try:
        import torchvision.transforms.functional_tensor as _ft
    except Exception:
        try:
            import torchvision.transforms.functional as _f
            mod_ft = types.ModuleType('torchvision.transforms.functional_tensor')
            mod_ft.rgb_to_grayscale = getattr(_f, 'rgb_to_grayscale', None)
            mod_ft.convert_image_dtype = getattr(_f, 'convert_image_dtype', None)
            sys.modules['torchvision.transforms.functional_tensor'] = mod_ft
        except Exception:
            mod_ft = types.ModuleType('torchvision.transforms.functional_tensor')
            def _rgb_to_grayscale(x):
                raise ImportError('rgb_to_grayscale not available')
            def _convert_image_dtype(x, dtype):
                raise ImportError('convert_image_dtype not available')
            mod_ft.rgb_to_grayscale = _rgb_to_grayscale
            mod_ft.convert_image_dtype = _convert_image_dtype
            sys.modules['torchvision.transforms.functional_tensor'] = mod_ft

    if 'torchvision.utils' not in sys.modules:
        mod_utils = types.ModuleType('torchvision.utils')
        def make_grid(x, nrow=8, padding=2, normalize=False, range=None, scale_each=False, pad_value=0):
            try:
                if isinstance(x, (list, tuple)) and len(x) > 0:
                    return x[0]
                return x
            except Exception:
                return x
        mod_utils.make_grid = make_grid
        sys.modules['torchvision.utils'] = mod_utils

    if 'realesrgan.version' not in sys.modules:
        vermod = types.ModuleType('realesrgan.version')
        vermod.__version__ = '0.3.0'
        vermod.__all__ = ['__version__']
        sys.modules['realesrgan.version'] = vermod

    def _infer_netscale(argv):
        for i, a in enumerate(argv):
            if a in ('-n', '--name', '--model') and i+1 < len(argv):
                m = re.match(r"(\\d+)x", argv[i+1])
                if m:
                    try:
                        return int(m.group(1))
                    except Exception:
                        pass
        for i, a in enumerate(argv):
            if a in ('-s', '--scale') and i+1 < len(argv):
                try:
                    return int(argv[i+1])
                except Exception:
                    pass
        return 4

    def _detect_weight_file():
        try:
            wdir = os.path.join(os.getcwd(), 'weights')
            if not os.path.isdir(wdir):
                return None
            exts = ('.pth', '.pt', '.safetensors')
            candidates = [f for f in os.listdir(wdir) if f.lower().endswith(exts)]
            if not candidates:
                return None
            for name in candidates:
                if '4x' in name.lower() or 'ultrasharp' in name.lower():
                    return os.path.join(wdir, name)
            return os.path.join(wdir, candidates[0])
        except Exception:
            return None

    if __name__ == '__main__':
        argv = sys.argv[1:]
        netscale = _infer_netscale(argv)
        detected_model = _detect_weight_file()
        script_path = os.path.join(os.getcwd(), 'inference_realesrgan.py')
        try:
            with open(script_path, 'r', encoding='utf-8') as _f:
                _code = _f.read()
        except Exception:
            sys.argv = [script_path] + argv
            runpy.run_path('inference_realesrgan.py', run_name='__main__')
        else:
            _globals = {'__name__': '__main__', '__file__': script_path, 'netscale': netscale, 'model': detected_model, 'outscale': netscale, 'scale': netscale}
            _globals['sys'] = sys
            sys.argv = [script_path] + argv
            exec(compile(_code, script_path, 'exec'), _globals)
    ''')
# --- helper funcs ---

def safe_run(cmd, timeout=3600):
    try:
        return subprocess.run(cmd, capture_output=True, text=True, timeout=timeout)
    except Exception as e:
        return type('R', (), {'returncode': 99, 'stdout': '', 'stderr': str(e)})()
# --- main flow (safe guard) ---
print('Auto-block status: confirm=', confirm, 'install=', install, 'run=', run, 'restart_kernel=', restart_kernel)
if not confirm:
    print('Run aborted: set confirm = True after reviewing flags and paths (this cell is safe by default).')
    print('\nPaths to check:')
    print('  WORKSPACE =', WORKSPACE)
    print('  REPO_DIR =', REPO_DIR)
    print('  SHIM_PATH =', SHIM_PATH)
    print('  INFERENCE_LOG =', INFERENCE_LOG)
    print('  frames_dir (input) =', frames_dir)
    print('  proposed run args =', run_args)
else:
    # 1) optionally install deps
    if install:
        print('Installing packages:', deps)
        r = safe_run([sys.executable, '-m', 'pip', 'install', '-q'] + deps, timeout=1800)
        print('pip exit', r.returncode)
    # 2) write shim to SHIM_PATH (try workspace, fallback to cwd)
    try:
        os.makedirs(os.path.dirname(SHIM_PATH), exist_ok=True)
        with open(SHIM_PATH, 'w', encoding='utf-8', newline='\n') as f:
            f.write(SHIM)
        py_compile.compile(SHIM_PATH, doraise=True)
        used_shim = SHIM_PATH
        print('Wrote shim to', SHIM_PATH)
    except Exception as e:
        print('Could not write to', SHIM_PATH, '-> falling back to', FALLBACK_SHIM, '; reason:', e)
        with open(FALLBACK_SHIM, 'w', encoding='utf-8') as f:
            f.write(SHIM)
        used_shim = FALLBACK_SHIM
        print('Wrote shim to fallback', FALLBACK_SHIM)
    # 3) ensure repo exists (clone if missing)
    if not os.path.exists(REPO_DIR):
        print('Real-ESRGAN repo missing; attempting to clone into', REPO_DIR)
        rc = safe_run(['git', 'clone', 'https://github.com/xinntao/Real-ESRGAN', REPO_DIR], timeout=900)
        print('git clone rc=', rc.returncode)
    # 4) copy candidate weight from common dataset mount (if present)
    candidate = os.path.join('/kaggle/input/comfyui-models-gojo', '4x-UltraSharp.pth')
    if os.path.exists(candidate) and os.path.exists(REPO_DIR):
        try:
            wd = os.path.join(REPO_DIR, 'weights')
            os.makedirs(wd, exist_ok=True)
            shutil.copy(candidate, wd)
            print('Copied weight into', wd)
        except Exception as e:
            print('Failed to copy weight:', e)
    # 5) prepare run_cmd using used_shim (shim inside repo preferred)
    if os.path.exists(used_shim):
        run_cmd = [sys.executable, used_shim] + run_args
    else:
        run_cmd = None
    print('Prepared run_cmd:', run_cmd)
    # 6) optionally run and capture logs
    if run and run_cmd:
        print('Executing run_cmd (timeout=3600s)...')
        r = safe_run(run_cmd, timeout=3600)
        print('Return code =', r.returncode)
        try:
            with open(INFERENCE_LOG, 'a', encoding='utf-8') as lf:
                lf.write('\n=== AUTO ATTEMPT STDOUT ===\n')
                lf.write(r.stdout or '')
                lf.write('\n=== AUTO ATTEMPT STDERR ===\n')
                lf.write(r.stderr or '')
        except Exception as _e:
            print('Could not write inference log:', _e)
    elif run:
        print('run requested but run_cmd is not prepared; check shim/repo')
    # 7) optionally request kernel restart (best effort)
    if restart_kernel:
        print('Attempting programmatic kernel restart (may require manual confirm in Kaggle UI)...')
        try:
            from IPython.display import display, Javascript
            display(Javascript('Jupyter.notebook.kernel.restart()'))
            print('Kernel restart requested. If notebook does не restart автоматически, пожалуйста используйте Kernel -> Restart in the UI.')
        except Exception as _e:
            print('Programmatic restart not available; please restart kernel manually via UI:', _e)

In [ ]:
# OPTIONAL: Install papermill & nbconvert in the current environment (run this in Kaggle if you need to run notebooks programmatically)
# Run only when you want to enable papermill-based execution (this may install packages and take a few minutes).
# Set confirm_install = True to actually run the install, otherwise this cell just prints инструкции.
confirm_install = False
if not confirm_install:
    print('To enable papermill execution, set confirm_install = True and run this cell; it will install papermill, nbconvert and jupyter_client.')
else:
    import subprocess, sys
    pkgs = ['papermill', 'nbconvert', 'jupyter_client']
    print('Installing:', pkgs)
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-q'] + pkgs)
    print('Installed papermill and related packages. You can now run papermill-based execution.\n           Example:')
    print('  python /apps/ComfyCloud_My_Work_Flow/run_notebook_safe.py <input.ipynb> <output.ipynb> --kernel python3')
```
